In [3]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Sequential
from keras import backend as K
from talos.model.early_stopper import early_stopper
import talos as ta
import numpy as np

params = {
    'conv1_filters':  (4, 32, 8),
    'conv1_ksize':    [3, 5, 7],
    'max_pool_1':     [2, 3, 4],
    'conv2_filters':  (4, 16, 4),
    'conv2_ksize':    [3, 5, 7],
    'max_pool_2':     [2, 3, 4],
    'conv3_filters':  (4, 16, 4),
    'conv3_ksize':    [3, 5, 7],
    'max_pool_3':     [2, 3, 4],
    'conv0_ksize':    [3, 5, 7], # decoder only
    'optimizer':      ['adadelta'],
    'batch_size':     [2048],
    'epochs':         [300]
}

# If we force our images dimensions to be powers of 2, hyperparameter tuning becomes dramatically simpler.
def ae_model(x_train, y_train, x_val, y_val, params):
    params = {k: int(v) if v.isnumeric() else v for k,v in params.items()}
    ae = Sequential([
      Conv2D(params['conv1_filters'], (params['conv1_ksize'], params['conv1_ksize']), input_shape=x_train.shape[1:], activation='relu', padding='same'),
      MaxPooling2D((params['max_pool_1'], params['max_pool_1']), padding='same'),
      Conv2D(params['conv2_filters'], (params['conv2_ksize'], params['conv2_ksize']), activation='relu', padding='same'),
      MaxPooling2D((params['max_pool_2'], params['max_pool_2']), padding='same'),
      Conv2D(params['conv3_filters'], (params['conv3_ksize'], params['conv3_ksize']), activation='relu', padding='same'),
      MaxPooling2D((params['max_pool_3'], params['max_pool_3']), padding='same'),

      Conv2D(params['conv3_filters'], (params['conv3_ksize'], params['conv3_ksize']), activation='relu', padding='same'),
      UpSampling2D((params['max_pool_3'], params['max_pool_3'])),
      Conv2D(params['conv2_filters'], (params['conv2_ksize'], params['conv2_ksize']), activation='relu', padding='same'),
      UpSampling2D((params['max_pool_2'], params['max_pool_2'])),
      Conv2D(params['conv1_filters'], (params['conv1_ksize'], params['conv1_ksize']), activation='relu', padding='same'),
      UpSampling2D((params['max_pool_1'], params['max_pool_1'])),
      Conv2D(1, (params['conv0_ksize'], params['conv0_ksize']), activation='sigmoid', padding='same')
    ])
    ae.compile(optimizer=params['optimizer'], loss='mean_absolute_error')
    out = ae.fit(x_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], verbose=2,
                 validation_data=[x_val, y_val], callbacks=early_stopper(params['epochs'], mode='strict'))
    return out, ae

/Users/pjrule/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/pjrule/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [1]:
import numpy as np
import os
frames = []

for map_frames in os.listdir('frames/wi'):
    print(map_frames)
    uncompressed = np.load(os.path.join('frames', 'wi', map_frames))
    frames += list(uncompressed['kwds'].reshape(1)[0].values())
 

a31002f494bf.npz
e65ddf30a0d6.npz
e65d06674ec5.npz
352aad0346f9.npz
5f413f9e2b13.npz
afca69875bae.npz
6dffbd562ede.npz
33923212ec06.npz
713e148d66cb.npz
779675b51a51.npz
1be0b3cfc81f.npz
6af6e45693e2.npz
0182ae74c8b0.npz
3084c24c266f.npz
4f4c2950f070.npz
8bccc215ef38.npz
ce67c9ccb408.npz
d6038b5dd437.npz
71768cc6cf35.npz
c2e7eaa8358a.npz
c78adc25f9f4.npz
c08bedfc58e4.npz
21d0def62efc.npz
052db0c448bb.npz
17d380d4b56e.npz
7699100b8751.npz
189f4d10e46f.npz
c35a3ef3e105.npz
3f557c27b56c.npz
23306e87e8f9.npz
02d1cdc3d7fb.npz
6fbf95329302.npz
d2b981403d0d.npz
29266e49ea13.npz
93c252db988c.npz
d7df6e5c2cc3.npz
556df7fadf8b.npz
148e1a7ed88c.npz
3b6164b9dbb8.npz
a0a88a99d929.npz
c672faddc6b7.npz
c0d587c0cda5.npz
21e543261cd9.npz
fb68418f9e93.npz
b148699b7b9e.npz
20a0c655dccc.npz
a0c4ad7b32a8.npz
c04a0a7a9e64.npz
8891ae21560a.npz
86be26636557.npz
07fe685ba184.npz
c9fb41e933a9.npz
6c07245f52e9.npz
67d62441111a.npz
0ad4abcf63f7.npz
b7f91748ac4f.npz
ec9465dc181b.npz
e9752ce3df45.npz
59d8c981289f.n

TypeError: list indices must be integers or slices, not tuple

In [4]:
frames = np.array(frames)[:,:256,:256]
h = ta.Scan(frames, frames, params=params, model=ae_model, dataset_name='wi_frames', experiment_no='1', grid_downsample=0.01)

KeyboardInterrupt: 

In [13]:
from scipy.sparse import load_npz
geo = load_npz('wi_geo_65536.npy.npz')

In [17]:
from math import sqrt
sqrt(65720)

256.35912310662945

In [20]:
import pickle
wi = pickle.load(open('wi_data/wi_160.pkl', 'rb'))
wi.reset()
wi.load_geo_mapping('wi_geo_65536.npy.npz')

/Users/pjrule/anaconda3/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [28]:
ALPHA = wi.alpha
print(ALPHA)

0.9340717826304465


In [31]:
import states
wi_256 = states.Wisconsin('../data/WI/ward_maps/2020/2012_2020_Election_Data_with_2017_Wards.shp', 
                          '../data/WI/vtd_elections.csv',
                          '../data/WI/vtd_demographics.csv',
                          160000,
                          round(256**2 * ALPHA))

KeyboardInterrupt: 